# XMM-Newton Event List Structure
<hr style="border: 2px solid #fadbac" />

- **Description:** A basic guide to the internal structure of an XMM-Newton event list.
- **Level:** Intermediate
- **Data:** A random XMM observation (obsid=0079570201).
- **Requirements:** Must be run using pySAS version 2.2.2 or higher.
- **Credit:** Ryan Tanner (October 2025)
- **Support:** <a href="https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html">XMM Newton GOF Helpdesk</a>
- **Last verified to run:** 17 October 2025, for SAS v22.1 and pySAS v2.2.2

<hr style="border: 2px solid #fadbac" />

## 1. Introduction

The event list is the main data product for X-ray astronomy. But, unless you get very deep into the weeds of X-ray analisys you will never actually interact directly with the contents of an event list.

This is a **very** basic field guide to the weeds for when you eventually go there. The XMM event list is a FITS file and can be accessed using `astropy`'s FITS file handling module, `astropy.io.fits`. You can find more information about that [at this link](https://docs.astropy.org/en/stable/io/fits/index.html).

#### Useful Links

- [`pysas` Documentation](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/pysas/index.html "pysas Documentation")
- [`pysas` on GitHub](https://github.com/XMMGOF/pysas)
- [Common SAS Threads](https://www.cosmos.esa.int/web/xmm-newton/sas-threads/ "SAS Threads")
- [Users' Guide to the XMM-Newton Science Analysis System (SAS)](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/sas_usg/USG/SASUSG.html "Users' Guide")
- [The XMM-Newton ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/ "ABC Guide")
- [XMM Newton GOF Helpdesk](https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html "Helpdesk") - Link to form to contact the GOF Helpdesk.

<div class="alert alert-block alert-warning">
    <b>Warning:</b> By default this notebook will place observation data files in your default <tt>data_dir</tt> directory. Make sure pySAS has been configured properly.
</div>

In [ ]:
# pySAS imports
import pysas
from pysas import MyTask

# Useful imports
import os

# Astropy imports
from astropy.io import fits

# To handle certain warnings
import warnings
warnings.filterwarnings("ignore")

We will start by downloading the ODFs for a single observation, calibrating it, and then running `epproc` to create a basic event list.

In [ ]:
obsid = '0079570201'
my_obs = pysas.ObsID(obsid)
my_obs.basic_setup(overwrite=False,
                   rerun=False,
                   run_emproc=False,
                   run_rgsproc=False)

## 2. Loading the Event List

After this is finished you should have a single event list for the pn.

In [ ]:
event_list = my_obs.files['PNevt_list'][0]
print(event_list)

Even though the file extension is `.ds`, it is in the FITS format. Let's load the event list and take a look at what is in there.

In [ ]:
hdu = fits.open(event_list)

## 3. Header Info

We can list the basic info for the FITS file using the `info` method. Here we see that the file contains 64 Header/Data Units (HDUs). 

In [ ]:
hdu.info()

The primary HDU contains all the basic information about the FITS file, what it contains, how it was created, and information on the observation.

In [ ]:
primary_header = hdu[0].header
primary_header

You can access the values like you would access values in a standard Python dictionary.

In [ ]:
primary_header['OBS_ID']

In [ ]:
primary_header['INSTRUME']

One of the keywords, `XPROC0`, contains the actual SAS command run by `epproc` to create the FITS file.

In [ ]:
primary_header['XPROC0']

## 4. The Primary HDU

Now let's take a look at the header for the first HDU after the primary HDU. This HDU contains the event data in the event list. 

In [ ]:
hdu[1].header

***
The number of rows in the table is the number of events in the event list.

In [ ]:
nrows = hdu[1].header['NAXIS2']
print(f'In this case there are {nrows} events in the event list.')

***
You can access the actual data in the following way:

In [ ]:
event_list_data = hdu[1].data
print(f'The data is stored in a list with {len(event_list_data)} elements in the list.') 

***
The header for the HDU contains the information for the columns, that is, all the information associated with each event. We can display the names of the columns like this:

In [ ]:
event_list_data.dtype

## 5. The Raw Event Data

We can look at the information for individual events by picking off a single element. For example the first element is,

In [ ]:
event_list_data[0]

#### This is the raw data for a single event in the event list.

## 6. Data Flags

In [ ]:
def filter_event_list(in_event_list,
                      filter,
                      filtered_event_list = 'filtered_event_list.fits'):

    # Filter expression
    expression = f'#{filter}'

    inargs = {'table'           : in_event_list, 
              'withfilteredset' : 'yes', 
              "expression"      : expression, 
              'filteredset'     : filtered_event_list, 
              'filtertype'      : 'expression', 
              'keepfilteroutput': 'yes', 
              'updateexposure'  : 'yes', 
              'filterexposure'  : 'yes'}
    
    MyTask('evselect', inargs, output_to_terminal = False).run()

    with fits.open(filtered_event_list) as hdu:
        nrows = hdu[1].header['NAXIS2']
        comment = hdu[1].header.comments[filter]
    print(f'There are {nrows} events in the event list with the {filter} ({comment}) flag.')

Not all of the events in the event list are valid events for what we need. In the header for the event HDU is a list of different flags that each event may have.
```
XMMEA_0 = '(FLAG & 0x1) != 0'  / INVALID_PATTERN                                
XMMEA_2 = '(FLAG & 0x4) != 0'  / CLOSE_TO_CCD_WINDOW                            
XMMEA_3 = '(FLAG & 0x8) != 0'  / ON_OFFSET_COLUMN                               
XMMEA_4 = '(FLAG & 0x10) != 0' / NEXT_TO_OFFSET_COLUMN                          
XMMEA_5 = '(FLAG & 0x20) != 0' / CLOSE_TO_ONBOARD_BADPIX                        
XMMEA_6 = '(FLAG & 0x40) != 0' / CLOSE_TO_BRIGHTPIX                             
XMMEA_8 = '(FLAG & 0x100) != 0' / CLOSE_TO_DEADPIX                              
XMMEA_11= '(FLAG & 0x800) != 0' / IN_SPOILED_FRAME                              
XMMEA_16= '(FLAG & 0x10000) != 0' / OUT_OF_FOV                                  
XMMEA_17= '(FLAG & 0x20000) != 0' / IN_BAD_FRAME                                
XMMEA_19= '(FLAG & 0x80000) != 0' / COSMIC_RAY                                  
XMMEA_20= '(FLAG & 0x100000) != 0' / MIP_ASSOCIATED                             
XMMEA_21= '(FLAG & 0x200000) != 0' / ON_BADPIX                                  
XMMEA_22= '(FLAG & 0x400000) != 0' / SECONDARY                                  
XMMEA_23= '(FLAG & 0x800000) != 0' / TRAILING                                   
XMMEA_25= '(FLAG & 0x2000000) != 0' / OUT_OF_CCD_WINDOW                         
XMMEA_EP= '(FLAG & 0xcfa0000) == 0' / Select good PN events
```

<div class="alert alert-block alert-info">
<b>Note:</b> There are a total of 31 possible flags of the form <tt>'XMMEA_N'</tt> with <tt>'N'</tt> being an integer from 0 to 30 (<tt>N = 31</tt> is no flag). The flag `XMMEA_EP` is actually a combination of a number of different flags. Only the flags listed in the header of the HDU are found in that file.
</div>

In [ ]:
with fits.open(event_list) as hduel:
    nrows = hduel[1].header['NAXIS2']
print(f'In total there are {nrows} events in the event list.\n')
filter_list = ['XMMEA_0','XMMEA_2','XMMEA_3','XMMEA_4','XMMEA_5',
               'XMMEA_6','XMMEA_8','XMMEA_11','XMMEA_16','XMMEA_17',
               'XMMEA_19','XMMEA_20','XMMEA_21','XMMEA_22','XMMEA_23',
               'XMMEA_25','XMMEA_EP']
for filter in filter_list:
    filter_event_list(event_list,filter)

***
For example, the events flagged with an 'INVALID_PATTERN' would make an image that looks like this:

In [ ]:
filter = 'XMMEA_0'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

***

Below are several more plots of different flags.

In [ ]:
filter = 'XMMEA_2'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

In [ ]:
filter = 'XMMEA_3'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

In [ ]:
filter = 'XMMEA_4'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

In [ ]:
filter = 'XMMEA_5'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

In [ ]:
filter = 'XMMEA_6'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

In [ ]:
filter = 'XMMEA_16'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

In [ ]:
filter = 'XMMEA_21'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1)

#### The "Good" Data

The `XMMEA_EP` flag marks the "good" data. This is what is used for typical data analysis.

In [ ]:
filter = 'XMMEA_EP'
filtered_event_list = 'filtered_event_list.fits'
filter_event_list(event_list,filter,filtered_event_list=filtered_event_list)
my_obs.quick_eplot(filtered_event_list,vmax=1000)

## 7. Conclusion

This is not a comprehensive guide to the XMM event list, but only a basic introduction of the structure of the event list. A full explanation of XMM data files can be [found at this link](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/dfhb/). Using `astropy`'s FITS module you can explore the event list in more detail.